In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
def process_file(file_path):
    save_path = file_path + '20240304/image' + file_path.split('/')[-1].replace('.csv', '.png')
    try:
        # 尝试自动检测分隔符，读取CSV文件
        data = pd.read_csv(file_path, sep=None, engine='python')
        # 过滤出包含实际电压和电流数据的行
        data_values = data[data.iloc[:, 0].str.startswith('DataValue')]
        # 分割字符串，转换数据类型
        voltage_current_data = data_values.iloc[:, 0].str.split(',', expand=True)
        voltage = pd.to_numeric(voltage_current_data[1].str.strip())
        current = pd.to_numeric(voltage_current_data[2].str.strip())
        # 绘图
        plt.figure(figsize=(10, 6))
        plt.plot(voltage, current, marker='o', linestyle='-', color='b')
        # 为每个点添加注释
        plt.title('Voltage vs. Current')
        plt.xlabel('Voltage (V)')
        plt.ylabel('Current (A)')
        plt.grid(True)
        # 保存图表
        plt.savefig(file_path.replace('.csv', '.png'))
        plt.close()
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 文件所在目录
directory = r"IV_TEST/20240312"

# 遍历目录中的文件
for file_name in os.listdir(directory):
    if file_name.endswith(".csv"):
        file_path = os.path.join(directory, file_name)
        process_file(file_path)


In [ ]:
directory = r"IV_TEST/20240304"
i=0
for file_name in os.listdir(directory):
    if file_name.endswith(".csv"):
        file_path = os.path.join(directory, file_name)
        print(file_path)
        i += 1

print(i)
        

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import os

# 你的CSV文件所在的目录
directory = r"IV_TEST/20240304"

# 创建Dash应用
app = dash.Dash(__name__)

# 获取CSV文件列表
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# 应用布局
app.layout = html.Div([
    dcc.Dropdown(
        id='file-dropdown',
        options=[{'label': f, 'value': os.path.join(directory, f)} for f in csv_files],
        value=None,
        placeholder="Select a file"
    ),
    dcc.Graph(id='voltage-current-graph')
])

# 回调函数，根据选择的文件名更新图表
@app.callback(
    Output('voltage-current-graph', 'figure'),
    [Input('file-dropdown', 'value')]
)
def update_graph(selected_file):
    if selected_file is None:
        return go.Figure()

    # 读取CSV文件并提取数据
    data = pd.read_csv(selected_file, sep=None, engine='python')
    data_values = data[data.iloc[:, 0].str.startswith('DataValue')]
    voltage_current_data = data_values.iloc[:, 0].str.split(',', expand=True)
    voltage = pd.to_numeric(voltage_current_data[1].str.strip())
    current = pd.to_numeric(voltage_current_data[2].str.strip())

    # 创建交互式图表
    fig = go.Figure(data=go.Scatter(x=voltage, y=current, mode='markers+lines', text=voltage_current_data[1] + ", " + voltage_current_data[2]))
    fig.update_layout(title='Voltage vs. Current', xaxis_title='Voltage (V)', yaxis_title='Current (A)')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
